In [1]:
from __future__ import print_function 

#%load_ext autoreload
#%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist,mnist_m
from data.tasks import *
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from experiments.models import *
from experiments.training import *
from util.kl import *
from util.misc import *

# Hyper-parameters
#batch_size = 128

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1*X GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024*4))])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)
    
#img_rows, img_cols = 32, 32
#project_folder = "/cephyr/users/adambre/Alvis/"

# # check output_dir, create it if not exists
#     if not os.path.isdir(output_dir):
#         os.makedirs(output_dir)

1 Physical GPUs, 1 Logical GPUs


# Load data & Train

In [3]:
#alphas=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
alphas=[0]#,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]#,0.3,0.5,0.7,0.9]
epsilons=[0.01]#,0.01,0.001]
architectures=["lenet"]
tasks=[4]
for t in tasks:
    x_source,y_source,x_target,y_target=load_task(t)
    print(len(y_source))
    print(len(y_target))
    for arch in architectures:
        for alpha in alphas:
            if alpha==0:
                pass
            else:
                print("Alpha is:"+str(alpha))
                x_bound, x_prior, y_bound , y_prior = train_test_split(x_source,y_source,test_size=alpha,random_state=69105)
                w_a=train_prior(alpha,1,x_prior,y_prior,x_target=x_target,y_target=y_target,save=True,task=t,binary=True,batch_size=128,architecture=arch)
            for epsilon in epsilons:
                w_s=train_posterior(alpha,x_source,y_source,None,x_test=x_source,y_test=y_source,save=True,epsilon=epsilon,task=t,binary=True,batch_size=128,architecture=arch)

mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 0.5223306 60.093555
---------------Load USPS----------------
Training set (7438, 32, 32, 3) (7438, 10)
Test set (1860, 32, 32, 3) (1860, 10)


70000
9298
Train on 70000 samples, validate on 70000 samples
70000/70000 [==============================] - 6s 81us/sample - loss: 0.1781 - accuracy: 0.9237 - val_loss: 0.0587 - val_accuracy: 0.9793
----------------------------------------
Finished training first posterior epoch

Epoch 00001: saving model to /cephyr/users/adambre/Alvis/mnist_transfer/posteriors/task4/Binary/lenet/10_0/2_1.ckpt

Epoch 00002: saving model to /cephyr/users/adambre/Alvis/mnist_transfer/posteriors/task4/Binary/lenet/10_0/2_2.ckpt

Epoch 00003: saving model to /cephyr/users/adambre/Alvis/mnist_transfer/posteriors/task4/Binary/lenet/10_0/2_3.ckpt

Epoch 00004: saving model to /cephyr/users/adam